In [1]:
import sys
import os
sys.path.append(os.path.join(os.path.dirname(""),".."))

import numpy
import pandas
from IPython.display import display
import custom
from konlpy.tag import Kkma as word_div
from tqdm import tqdm
import threading

In [2]:
class Preprocessor() :
    def __init__(self) :
        self._result = None
        self._error = None
    def _process_with_timeout(self, s : str, timeout : int) :
        def work() :
            try :
                self._result = word_div().morphs(s)
            except Exception as e :
                self._error = e        
        self._error = None
        thread = threading.Thread(target=work, daemon=True)
        thread.start()
        thread.join(timeout=timeout)

        if thread.is_alive() :
            raise Exception(f"Error : Time Exceeded")
        if self._error is not None :
            raise Exception(str(self._error))

        return self._result
    def process(self, s : str, timeout : int = 10) :
        result = self._process_with_timeout(s, timeout)
        return result


In [3]:
print(Preprocessor().process("안녕하세요"))
# print(Preprocessor().process("진짜 조낸 재밌다 굿굿굿굿굿굿굿굿굿굿굿굿굿굿굿굿굿굿굿굿굿굿"))

['안녕', '하', '세요']


In [4]:
original_df = pandas.read_csv("data/ratings_train.txt", sep = "\t", encoding = "UTF8")

display(original_df)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [5]:
df = original_df[['document','label']]

for i in tqdm(range(len(df))) :
    text = custom.text_preprocess_kor(str(df.iloc[i,0]), end_mark = True)
    if text == " " :
        df.iloc[i,0] = text
        continue
    try :
        df.iloc[i,0] = " ".join(Preprocessor().process(text))
    except Exception as e :
        df.iloc[i,0] = " "
        print(str(e), f"문장 : {text}")
    
word_len = []
for i in range(len(df)) :
    word_len.append(len(str(df.iloc[i,0]).split()))
df["word_len"] = word_len

df = df.drop(df[df["word_len"] == 0].index, axis = 0)

display(df)


 52%|█████████████████████████████████████                                   | 77289/150000 [38:28<12:55:31,  1.56it/s]

Error : Time Exceeded 문장 : 숨은글씨 찾기 의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리니의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리의리


 69%|█████████████████████████████████████████████████▌                      | 103164/150000 [52:56<7:22:11,  1.77it/s]

Error : Time Exceeded 문장 : 야야야야야야야야야야야야야야야야야야야야야야야야야야야야야


100%|████████████████████████████████████████████████████████████████████████| 150000/150000 [1:18:27<00:00, 31.86it/s]


,document,label,word_len
0,아 아 더빙 . 진짜 짜증나 네요 목소리,0,8
1,흠 . 포스터 보고 초 딩 영화 줄 . 오버 연기 조차 가볍 지 않 구나,1,16
2,너무 재 밓 어 었 다 그래서 보 는 것 을 추천 하 ㄴ다,0,14
3,교도소 이야기 구 멀 ㄴ . 솔직히 재미 는 없 다 . 평점 조정,0,14
4,사이몬페그 의 익살 스럽 ㄴ 연기 가 돋보이 었 더 ㄴ 영화 ! 스파이 더 맨 에서...,1,36
...,...,...,...
149995,인간 이 문제지 . 소 는 뭔 죄 이 ㄴ가 .,0,11
149996,평점 이 너무 낮 아서 .,1,6
149997,이것 이 뭐 이 요 ? 한국 인은 거들먹거리 고 필리핀 혼혈 은 착하 다 ?,0,16
149998,청춘 영화 의 최고 봉 . 방황 과 우울하 었 더 ㄴ 날 들 의 자화상,1,16


In [6]:

df.to_csv("data/korean_movie_train.txt", sep = "\t", encoding = "UTF8")

In [7]:
print(df["word_len"].max())
print(df["word_len"].mean())
print(df["word_len"].std())

110
19.13843220708776
16.248429319582733
